# Guia #1

In [ ]:
import numpy as np
import pandas as pd
from fractions import Fraction

data = pd.read_excel('Data/Ejercicio #1.xlsx')
data2 = pd.read_excel('Data/Ejercicio #2.xlsx')
data

### Machetes

In [ ]:
############### MACHETE ###############

# Media (datos agrupados)
# Me = (Xi*fi)/n # Intervalo de clase por frecuencia /n

# Moda (datos agrupados)
# Mo = Linf + a*fni/(fni+fii) # fni/fii: fni anterior/fii siguiente
# Mo = Linf + 5*np.divide(1, 1+2) # = 106.67

# Mediana (datps agrupados)
# OAM = n*.5 # Quiero la Mediana # Ojo que tira error si es la primer clase (Fni==0)
# Me = Linf + a*np.divide(OAM-Fni, fi) # Fni: Freciencia Acumulada Anterior | fi: Frecuencia Simple
# Me = 105 + 5*np.divide(7-4, 5) # = 108


# data = np.array([3, 4, 7, 41, 65, 74, 114])
# Mediana (datos sin agrupar)
# OAP = (n+1)*.5 # Busco la mediana
# Me = X(OAP) = X(4) # Cuando el modulo de OAP es 0 (OAP%2 == 0)

# Cuando cae entre dos clases, le sumo a la clase primera el porcentual de la diferencia entre ambas clases
# Me = X(OAP-1) + (OAP%1) * [X(OAP)-X(OAP-1)] # X(3) + RESTO*[X(4)-X(3)]

# Caluclo Varianza
# S*S = np.sum(X*X)/n - Me*Me
# CV = S/Me

### Funciones Auxiliares y Ejemplos

In [ ]:
# Ejemplo de la guia

ej = np.array([3, 4, 7, 41, 65, 74])
print('Ejercicio Paises-Covid:\n', ej)
n = ej.size
rate = .75 # Q3
OAP = (n+1)*rate
prev_class = int(OAP//1-1) # Caluclo la parte enetera y le resto 1 (indices comiencan en 0)
next_class = prev_class + 1

ej[prev_class] + (OAP%1)*(ej[next_class] - ej[prev_class])

In [ ]:
# Funcion que genera Tabla de Frecuencias n, y la media: Recibe DataFrame con Xi, fi

# Funcion que calcula el n es lo primero
def calcular_n(data_):
    result = data_.fi.sum()
    return result

# Funcion que genera el DF
def generar_tabla_frecuencias(data_):
    # Primero genero Xi * fi
    display(data_)
    Xi_fi = data_.Xi * data_.fi
    Xi_fi.name = 'Xi_fi'
    
    # Genero frecuencias acumuladas
    x_ant = 0
    Fi = []
    for x in data_.fi:
        Fi.append(x+x_ant)
        x_ant += x
    Fi = pd.Series(Fi, name='Fi')
    
    # Genero Serie Frecuencia Relativa y Frecuencia Acumulada
    fri = data_.fi.apply(lambda x: Fraction(x, n))
    fri = pd.Series(fri, name='fri')
    Fri = Fi.apply(lambda x: Fraction(x, n))
    Fri = pd.Series(Fri, name='Fri')
    
    # Concateno y devuelvo
    result = pd.concat([data_,Xi_fi, Fi, fri, Fri], axis=1)
    return result

data = generar_tabla_frecuencias(data)
data

In [ ]:
# Funcion de calculo de la media
def calcular_media(data_):
    result = (data_.Xi * data_.fi).sum()/n
    return result.round(2)

media = calcular_media(data)
media

In [ ]:
# Funcion que devuelve la mediana
def calcular_mediana(data_):
    OAM = n/2
    i=0 
    while data_.Fi[i]<OAM:
         i+=1
    
    # Si esta justo sobre el limite, le sumo la mitad
    if(data_.Fi[i]==OAM):
        i+=.5
    return i

mediana = calcular_mediana(data)
mediana

In [ ]:
# Calcula la Varainza
def calcular_varianza(data_):
    X2_fi = pd.Series(data_.Xi*data_.Xi*data_.fi, name='X2_fi')
    result = X2_fi.sum()/n - media*media
    return result

varianza=calcular_varianza(data)
varianza

In [ ]:
# Calcula el Desvio
def calcular_desvio(data_):
    return np.sqrt(calcular_varianza(data_)).round(2)

desvio = calcular_desvio(data)
desvio

In [ ]:
# Calcular Coeficiente Variacion
def calcular_coeficiente_variacion(data_):
    return np.divide(calcular_desvio(data_), media).round(2)

CV = calcular_coeficiente_variacion(data)
CV

In [ ]:
# Funcion que devuelve el percentil especificado
def calcular_percentil(data_, percentil_):
    OAP=percentil_*n/100
    i=0 
    while data_.Fi[i]<OAP:
         i+=1
    
    # Si esta justo sobre el limite, le sumo la mitad
    if(data_.Fi[i]==OAP):
        i+=.5
    return i

percentil = calcular_percentil(data, 25)
percentil

In [ ]:
def generar_tabla_frecuencias_intervalos(data_):
    data_ = data2 # COMENTAR ESTA LINEA!!!!
    Xi = pd.Series((data_.Xinf + data_.Xsup)/2, name='Xi')
    Xi_fi = pd.Series(Xi*data_.fi, name='Xi_fi')
    
    # Para el acumulado
    x_ant = 0
    Fi = []
    for x in data_.fi:
        x_ant+=x
        Fi.append(x_ant)
    Fi = pd.Series(Fi, name='Fi')
    
    # Concateno
    result = pd.concat([data_, Fi, Xi, Xi_fi], axis=1)
    return result

data = generar_tabla_frecuencias_intervalos(data)
data

In [ ]:
def calcular_media_intervalo(data_):
    return np.round(data_.Xi_fi.sum()/n, 2)

media_intervalo = calcular_media_intervalo(data)
media_intervalo

In [ ]:
def calcular_intervalo_modal_id(data_):
    moda = data_.fi.max()
    i=0
    while data_.fi[i]!=moda:
        i+=1
    return i
intervalo_modal_id = calcular_intervalo_modal_id(data)

def calcular_intervalo_modal(data_):
    return data_.iloc[calcular_intervalo_modal_id(data_)]

intervalo_modal = calcular_intervalo_modal(data)

print('ID:', intervalo_modal_id)
print(intervalo_modal)

In [ ]:
def calcular_moda_intervalo(data_):
    # Primero encuentro el intervalo modal
    intervalo_modal_id = calcular_intervalo_modal_id(data_)
    intervalo_modal = calcular_intervalo_modal(data_)
    
    # Genero las variables
    amplitud = intervalo_modal.Xsup - intervalo_modal.Xinf
    Xinf = intervalo_modal.Xinf
    d1 = data_.iloc[intervalo_modal_id].fi - data_.iloc[intervalo_modal_id-1].fi
    d2 = data_.iloc[intervalo_modal_id].fi - data_.iloc[intervalo_modal_id+1].fi
    print(amplitud, Xinf, d1, d2)
    
    # Aplico la formula
    return np.round(Xinf+amplitud*d1/(d1+d2), 2)

moda_intervalo = calcular_moda_intervalo(data)
moda_intervalo

In [ ]:
# Tambien se puede trabajar con un percentil de 50??????

# def calcular_intervalo_mediano_id(data_):
#     i=0
#     while data_.Fi[i]<n/2:
#         i+=1
#     return i

# intervalo_mediano_id=calcular_intervalo_mediano_id(data)

# def calcular_intervalo_mediano(data_):
#     return data_.iloc[calcular_intervalo_mediano_id(data_)]

# intervalo_mediano = calcular_intervalo_mediano(data)

# print('ID:', intervalo_mediano_id)
# print(intervalo_mediano)

In [ ]:
# Tambien se puede trabajar con un percentil de 50???????

# def calcular_mediana_intervalo(data_):
#     intervalo_mediano_id = calcular_intervalo_modal_id(data_)
#     intercalo_mediano = calcular_intervalo_mediano(data_)
    
#     OAM = n/2
#     amplitud = intervalo_mediano.Xsup - intervalo_mediano.Xinf
#     Xinf = intervalo_mediano.Xinf
#     fm = data_.iloc[intervalo_mediano_id].fi
    
#     # Si no hay acumulada anterior
#     if (intervalo_mediano_id == 0):
#         Fm_1=0
#     else:
#         Fm_1 = data_.iloc[intervalo_mediano_id-1].Fi
        
#     print(OAM, amplitud, Xinf, fm, Fm_1)
   
#     return np.round(Xinf+amplitud*(OAM-Fm_1)/fm, 2)

# mediana_intervalo = calcular_mediana_intervalo(data)
# mediana_intervalo

In [ ]:
percentil = 50 # 50 = Mediana

def calcular_intervalo_percentil_id(data_, percentil_):
    i=0
    while data_.Fi[i]<percentil_*n/100:
        i+=1
    return i

intervalo_percentil_id = calcular_intervalo_percentil_id(data, percentil)

def calcular_intervalo_percentil(data_, percentil_):
    return data_.iloc[calcular_intervalo_percentil_id(data_, percentil_)]

intervalo_percentil = calcular_intervalo_percentil(data, percentil)

print('ID:', intervalo_percentil_id)
intervalo_percentil

In [ ]:
def calcular_percentil_intervalo(data_, percentil_):
    intervalo_percentil_id = calcular_intervalo_percentil_id(data_, percentil_)
    intervalo_percentil = calcular_intervalo_percentil(data_, percentil_)
    
    OAP = percentil_*n/100
    amplitud=intervalo_percentil.Xsup - intervalo_percentil.Xinf
    Xinf = intervalo_percentil.Xinf
    fk = data_.iloc[intervalo_percentil_id].fi
    
    # Pregunto si es el la primer clase, ya que su acumulada seria igual a 0
    if(intervalo_percentil_id==0):        Fk_1 = 0
    else:
        Fk_1 = data_.iloc[intervalo_percentil_id-1].Fi
    
    print(OAP, n,  amplitud, Xinf, fk, Fk_1)
    
    return np.round(Xinf+amplitud*(OAP-Fk_1)/fk, 2)
    
percentil_intervalo = calcular_percentil_intervalo(data, percentil)
percentil_intervalo

In [ ]:
# Alternativa sin tener que usar Mediana
def calcular_intervalo_mediano_id(data_):
    calcular_intervalo_percentil_id(data_, 50)
intervalo_mediano_id=calcular_intervalo_mediano_id(data)
# intervalo_mediano_id

def calcular_intervalo_mediano(data_):
    calcular_intervalo_percentil(data_, 50)
intervalo_mediano = calcular_intervalo_mediano(data)
intervalo_mediano

In [ ]:
def calcular_mediana_intervalo(data_):
    calcular_percentil_intervalo(data_, 50)
mediana_intervalo = calcular_mediana_intervalo(data)
mediana_intervalo

## Ejercicios de la guia practica

#### Ejercicio #1

In [ ]:
# Ejercicio #1

data = pd.read_excel('Data/Ej1.xlsx')

display(data.head())
display(data.describe())
print(data.Salario.value_counts().head())

In [ ]:
distancia_IQ = data.describe().loc['75%'] - data.describe().loc['25%']
print('IQ3 - IQ1:', distancia_IQ)

#### Ejercicio #2

In [ ]:
data = pd.read_excel('Data/Ej2.xlsx')
display(data.head())
display(data.describe())
print(data.mode())

#### Ejercicio #3

In [ ]:
data = pd.read_excel('Data/Ej3.xlsx', index_col='Ano')
data

In [ ]:
superficie_serie = data.loc['Superficie']
produccion_serie = data.loc['Produccion']

print('Superficie:\n', superficie_serie.describe(), '\n')
print(superficie_serie.value_counts().head())
print('Rango:', superficie_serie.max() - superficie_serie.min())

In [ ]:
CV_superficie = superficie_serie.std() / superficie_serie.mean()
CV_produccion = np.divide(produccion_serie.std(), produccion_serie.mean())

print('Coeficiente Variacion Superficie:', CV_superficie.round(3))
print('Coeficiente Variacion Produccion:', CV_produccion.round(3))

#### Ejercicio #4

In [ ]:
data = pd.read_excel('Data/Ej4.xlsx')
data

In [ ]:
# Completo con las frecuencias

data = generar_tabla_frecuencias_intervalos(data)
display(data)

In [ ]:
# Intervalo Mediano

def agregar_mediana_columns(data_):
    result = data_.shape[0]
    return result



agregar_mediana_columns(data)

In [ ]:
############### MACHETE ###############

# Media (datos agrupados)
# Me = (Xi*fi)/n # Intervalo de clase por frecuencia /n

# Moda (datos agrupados)
# Mo = Linf + a*fni/(fni+fii) # fni/fii: fni anterior/fii siguiente
# Mo = Linf + 5*np.divide(1, 1+2) # = 106.67

# Mediana (datps agrupados)
# OAM = n*.5 # Quiero la Mediana # Ojo que tira error si es la primer clase (Fni==0)
# Me = Linf + a*np.divide(OAM-Fni, fi) # Fni: Freciencia Acumulada Anterior | fi: Frecuencia Simple
# Me = 105 + 5*np.divide(7-4, 5) # = 108


# data = np.array([3, 4, 7, 41, 65, 74, 114])
# Mediana (datos sin agrupar)
# OAP = (n+1)*.5 # Busco la mediana
# Me = X(OAP) = X(4) # Cuando el modulo de OAP es 0 (OAP%2 == 0)

# Cuando cae entre dos clases, le sumo a la clase primera el porcentual de la diferencia entre ambas clases
# Me = X(OAP-1) + (OAP%1) * [X(OAP)-X(OAP-1)] # X(3) + RESTO*[X(4)-X(3)]

# Caluclo Varianza
# S*S = np.sum(X*X)/n - Me*Me
# CV = S/Me